### 使用Runnable来连接多链结构

In [1]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi

prompt1=ChatPromptTemplate.from_template("{person}来自哪个城市？")
prompt2=ChatPromptTemplate.from_template("{city}属于哪个省份？用{language}来回答。")

llm = ChatTongyi(model="qwen-plus")

In [2]:
chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {
        "city":chain1,
        "language":itemgetter("language")
    }
    | prompt2
    | llm
    | StrOutputParser()
)

In [3]:
chain2.invoke({"person":"杨幂","language":"中文"})

'杨幂出生于中国北京市，北京市属于**直辖市**，不隶属于任何省份，但它位于**华北地区**，从行政划分上来看，北京市是中国省级行政区之一。所以，如果要回答省份的话，可以用：**北京市**。'

In [10]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt1=ChatPromptTemplate.from_template("生成一个{attribute}属性的颜色，除了返回这个颜色的名字以外不要做其他事。")
prompt2=ChatPromptTemplate.from_template("什么水果是这个颜色：{color}？只返回这个水果的名字，不要做其他事情。")
prompt3=ChatPromptTemplate.from_template("哪个国家的国旗有这种颜色：{color}？只返回这个国家的名字，不要做其他事情。")
prompt4=ChatPromptTemplate.from_template("有这个颜色的水果是{fruit}，有这个颜色的国旗的国家是{country}。")

llm_parser = llm | StrOutputParser()

color_generator = (
    RunnablePassthrough() 
    | RunnableLambda(lambda x: {"attribute": x})
    | prompt1
    | llm_parser
    | RunnableLambda(lambda color: {"color": color})
)

color2fruit = (
    RunnableLambda(lambda data: {"color": data["color"]})
    | prompt2
    | llm_parser
)

color2country = (
    RunnableLambda(lambda data: {"color": data["color"]})
    | prompt3
    | llm_parser
)

question_generator = (
    color_generator
    | {
        "fruit": color2fruit,
        "country": color2country,
        "color": RunnablePassthrough()
    }
    | prompt4
    | llm_parser
)

In [11]:
question_generator.invoke({"attribute": "鲜艳"})

'你说的是红色吧！苹果确实有红色的品种，比如红富士、嘎啦苹果等，它们的表皮呈现出鲜艳的红色。同时，中国国旗——五星红旗的底色也是红色，象征着革命和胜利。\n\n如果你指的是其他颜色，请进一步说明哦！不过从你的描述来看，红色是非常贴切的答案。红色不仅是中国国旗的颜色，也是许多红苹果的代表色，同时还在中国文化中有着吉祥、喜庆的寓意。'

In [16]:
# 唯物辩证链

planner = (
    ChatPromptTemplate.from_template("生成一个关于{input}的论点")
    | llm
    | StrOutputParser()
    | {"base_response":RunnablePassthrough()}
)

arguments_support = (
    ChatPromptTemplate.from_template("列出以下内容的优点或积极方面：{base_response}")
    | llm
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template("列出以下内容的缺点或消极方面：{base_response}")
    | llm
    | StrOutputParser()
)

final_response = (
    ChatPromptTemplate.from_messages(
        [
            ("system","根据评论生成最终的回复"),
            ("ai","{original_response}"),
            ("human","积极：\n{result_1}\n\n消极：{result_2}"),
        ]
    )
    | llm
    | StrOutputParser()
)

chain = (
    planner
    | {
        "result_1":arguments_support,
        "result_2":arguments_against,
        "original_response":itemgetter("base_response")
    }
    | final_response
)

In [17]:
chain.invoke({"input":"不婚主义"})

'感谢您的分享，以下是对不婚主义积极与消极两方面分析的综合回复：\n\n---\n\n### **综合回复：**\n\n不婚主义作为一种个人选择，既体现了现代社会对个体自由和多元价值的尊重，也带来了新的挑战和思考。从积极层面来看，不婚主义突破了传统婚姻观念的束缚，允许人们根据自身需求和价值观决定是否进入婚姻。它不仅减轻了经济与情感压力，还挑战了性别角色和社会规范，为个体提供了更大的灵活性和自主权。此外，不婚主义推动了社会观念的进步，促使我们重新审视婚姻的意义，并接纳更多元的生活方式。\n\n然而，不婚主义也存在一些潜在的消极影响。在社会文化层面，不婚者可能面临来自家庭、亲友或社会的传统观念压力，甚至遭遇孤立感或代际冲突。在经济养老方面，不婚可能导致长期经济保障不足以及老年阶段缺乏亲密伴侣支持的问题。同时，长期单身可能增加孤独感，缩小社会关系网络，对情感健康产生一定影响。此外，不婚主义的普及还可能间接导致生育率下降，从而加剧人口老龄化问题，给国家经济发展和社会资源分配带来挑战。\n\n综上所述，不婚主义是一种值得尊重的个人选择，但其背后的社会、经济和心理影响也需要被充分关注。为了更好地支持不婚主义者以及其他多样化的生活方式，我们需要构建更加包容的社会环境，完善相关政策体系（如养老保障、医疗支持等），并鼓励公众以开放的态度看待不同的人生选择。只有这样，每个人才能在追求幸福的过程中找到最适合自己的道路。\n\n--- \n\n希望以上回复能为您提供全面而平衡的观点！'

In [ ]:
# 查询SQL

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase

template="""基于上面的数据表，写一个可以回答用户提问的sql查询语句：{schema}\n问题：{question}\nSql Query:"""

prompt = ChatPromptTemplate.from_template(template)

db = SQLDatabase.from_uri("sqlite:...")

db.get_table_info()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models.tongyi import ChatTongyi

llm = ChatTongyi(model="qwen-plus")

def get_schema(_):
    return db.get_table_info()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

sql_response.invoke({"question":"...?"})

In [ ]:
def run_query(query):
    print(query)
    return db.run(query)

template = """Based on the table schema below,question,sql query and sql
{schema}

Question:{question}
Sql Query:{query}
Sql Response:{response}"""

prompt_response = ChatPromptTemplate.from_template(template)

full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(schema=get_schema,response=lambda x: db.run(x["query"]))
    | prompt_response
    | llm
)

full_chain.invoke({"question":"...?"})

In [19]:
# 自定义输出解释器  一个python编程助手

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.utilities import PythonREPL
from langchain_community.chat_models.tongyi import ChatTongyi

template = """根据用户的需求帮助用户编写python代码，只需要返回markdown格式的python代码，比如：
```python
....
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",template),
        ("human","{input}")
    ]
)

llm = ChatTongyi(model="qwen-plus",temperature=0)

def _sanitize_output(text:str):
    _, after = text.split("```python")
    return after.split("```")[0]

chain = prompt | llm | StrOutputParser() | _sanitize_output | PythonREPL().run

In [21]:
chain.invoke({"input":"找出10里面所有质数"})

'[2, 3, 5, 7]\n'